In [ ]:
# 1. INSTALAR LIBRERÍAS
!pip install transformers datasets accelerate protobuf -Uq

import os
import time

print("✅ Librerías instaladas.")
print("🔄 REINICIANDO KERNEL PARA APLICAR CAMBIOS...")
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 112.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211

In [2]:
import os

# CONFIGURACIÓN DE RUTAS
# Ajusta el nombre si Kaggle le puso otro, pero según tu captura es este:
BASE_PATH = "/kaggle/input/ascii-architect-v1-2"

FILES = {
    "DIAMOND":  f"{BASE_PATH}/dataset_diamonds.jsonl",
    "CYLINDER": f"{BASE_PATH}/dataset_cylinders.jsonl"
}

print("🕵️ INSPECCIONANDO RUTAS...")
all_good = True
for name, path in FILES.items():
    if os.path.exists(path):
        print(f"   ✅ {name}: ENCONTRADO -> {path}")
    else:
        print(f"   ❌ {name}: NO EXISTE en {path}")
        all_good = False

if all_good:
    print("\n🚀 DATOS LOCALIZADOS. LISTO PARA ENTRENAR.")
else:
    print("\n⚠️ Revisa el panel derecho para ver el nombre real de la carpeta.")

🕵️ INSPECCIONANDO RUTAS...
   ✅ DIAMOND: ENCONTRADO -> /kaggle/input/ascii-architect-v1-2/dataset_diamonds.jsonl
   ✅ CYLINDER: ENCONTRADO -> /kaggle/input/ascii-architect-v1-2/dataset_cylinders.jsonl

🚀 DATOS LOCALIZADOS. LISTO PARA ENTRENAR.


In [3]:
import torch
from transformers import GPT2Tokenizer

MODEL_CHECKPOINT = "gpt2"

# DICCIONARIO MAESTRO V18 (EXPANDIDO)
SPECIAL_TOKENS = {
    "pad_token": "[PAD]",
    "additional_special_tokens": [
        "[START]", "[STOP]", "[VOID]", 
        "░", 
        "<L01>", "<L02>", "<L03>", "<L04>", "<L05>", "<L06>", "<L07>", "<L08>", "<L09>", "<L10>", "<L11>", "<L12>", "<L13>", "<L14>", "<L15>",
        "[S:00]", 
        "N", "S", "E", "W", "+", "-", "|", "^", "v", "<", ">", "/", "\\", ".", "'",
        # TIPOS NUEVOS
        "[TYPE:BOX]", "[TYPE:ARROW]", "[TYPE:DIAMOND]", "[TYPE:CYLINDER]",
        "[STYLE:SOLID]", "[STYLE:DB]",
        "[DIR:UP]", "[DIR:DOWN]", "[DIR:LEFT]", "[DIR:RIGHT]", 
        "[DIM:", "[LEN:", "[SIZE:"
    ]
}

print("⚙️ Configuración V18 cargada con soporte para Rombos y Cilindros.")

⚙️ Configuración V18 cargada con soporte para Rombos y Cilindros.


In [5]:
import gc
from datasets import load_dataset
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

def train_expert(expert_name, jsonl_path):
    print(f"\n▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
    print(f"💎 CONSTRUYENDO EXPERTO: {expert_name}")
    print(f"▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
    
    output_dir = f"./expert_{expert_name.lower()}"
    
    # 1. Tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_CHECKPOINT)
    tokenizer.add_special_tokens(SPECIAL_TOKENS)
    
    # 2. Dataset
    dataset = load_dataset("json", data_files=jsonl_path, split="train")
    
    def format_fn(x):
        text = f"{x['prompt']} {x['completion']}"
        return tokenizer(text, truncation=True, padding="max_length", max_length=150)
    
    tokenized = dataset.map(format_fn, batched=False, remove_columns=dataset.column_names)
    
    # 3. Modelo
    model = GPT2LMHeadModel.from_pretrained(MODEL_CHECKPOINT)
    model.resize_token_embeddings(len(tokenizer))
    
    # 4. Argumentos (Overfitting para precisión)
    args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=32,
        learning_rate=5e-4,
        save_strategy="no",
        logging_steps=50,
        report_to="none"
    )
    
    trainer = Trainer(model=model, args=args, train_dataset=tokenized, data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False))
    trainer.train()
    
    # 5. Guardar
    print(f"💾 Guardando {expert_name}...")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # 6. Limpiar
    del model, trainer, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# --- EJECUTAR ---
if os.path.exists(FILES["DIAMOND"]):
    train_expert("DIAMOND", FILES["DIAMOND"])

if os.path.exists(FILES["CYLINDER"]):
    train_expert("CYLINDER", FILES["CYLINDER"])

print("\n✨ ENTRENAMIENTO DE NUEVOS EXPERTOS COMPLETADO.")

2025-12-14 21:48:56.951571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765748937.139241     109 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765748937.195833     109 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

💾 Guardando DIAMOND...

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
💎 CONSTRUYENDO EXPERTO: CYLINDER
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Step,Training Loss
50,1.858500
100,0.218500
150,0.108300
200,0.079500
250,0.056400
300,0.051000
350,0.048500
400,0.047700
450,0.046600
500,0.046800


💾 Guardando CYLINDER...

✨ ENTRENAMIENTO DE NUEVOS EXPERTOS COMPLETADO.


In [6]:
from transformers import pipeline

def test_model(name, prompt):
    path = f"./expert_{name.lower()}"
    print(f"\n🧪 PROBANDO {name}...")
    try:
        gen = pipeline("text-generation", model=path, tokenizer=path, device=0)
        out = gen(prompt, max_length=150, pad_token_id=50256, do_sample=False)
        print(out[0]['generated_text'].replace("░", " ").replace(prompt, ""))
        del gen
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"Error: {e}")

# Test Rombo (Tamaño 4)
test_mod

NameError: name 'test_mod' is not defined

In [7]:
import shutil
from IPython.display import FileLink

zip_name = "ASCII_Architect_V2_Expansion"
os.makedirs("Expansion_Pack", exist_ok=True)

# Mover carpetas
for folder in ["expert_diamond", "expert_cylinder"]:
    if os.path.exists(folder):
        shutil.move(folder, f"Expansion_Pack/{folder}")

# Zipear
shutil.make_archive(zip_name, 'zip', "Expansion_Pack")

print(f"✅ PAQUETE DE EXPANSIÓN LISTO:")
FileLink(f"{zip_name}.zip")

✅ PAQUETE DE EXPANSIÓN LISTO:


/kaggle/working/ASCII_Architect_V2_Expansion.zip

In [9]:
from transformers import pipeline
import torch
import re
import os

def test_architect_expert(name, prompt):
    # CORRECCIÓN: Buscamos dentro de Expansion_Pack porque ya los moviste ahí
    folder_name = f"expert_{name.lower()}"
    path_root = f"./{folder_name}"
    path_pack = f"./Expansion_Pack/{folder_name}"
    
    # Detectar dónde demonios están
    if os.path.exists(path_root):
        path = path_root
    elif os.path.exists(path_pack):
        path = path_pack
    else:
        print(f"❌ NO ENCUENTRO EL MODELO '{folder_name}'. ¿Se borró?")
        return

    print(f"\n========================================")
    print(f"💎 PROBANDO EXPERTO: {name}")
    print(f"📂 Desde: {path}")
    print(f"========================================")
    
    try:
        # Cargar Pipeline
        gen = pipeline("text-generation", model=path, tokenizer=path, device=0)
        
        # Generar
        out = gen(prompt, max_length=200, pad_token_id=50256, do_sample=False)
        
        # Limpieza
        raw_text = out[0]['generated_text']
        content = raw_text.replace(prompt, "").replace("[STOP]", "")
        content = content.replace("░", " ")
        content = re.sub(r'<L\d+>', '', content)
        content = re.sub(r'\[S:\d+\]', '', content)
        
        print(content)
        
        del gen
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"❌ Error: {e}")

# --- BATERÍA DE PRUEBAS ---
test_architect_expert("DIAMOND", "[TYPE:DIAMOND] [STYLE:SOLID] [SIZE:4]")
test_architect_expert("CYLINDER", "[TYPE:CYLINDER] [STYLE:DB] [DIM:10x5]")

Device set to use cuda:0



💎 PROBANDO EXPERTO: DIAMOND
📂 Desde: ./Expansion_Pack/expert_diamond


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   
  
  
  
    
  
     
  
     
  
  
  

💎 PROBANDO EXPERTO: CYLINDER
📂 Desde: ./Expansion_Pack/expert_cylinder


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
